In [1]:
import random
import json
import pickle
import numpy as np


In [2]:
import nltk
import tensorflow as tf
from nltk.stem import WordNetLemmatizer


In [3]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())


In [4]:
words = [ ]
classes = [ ]
document = [ ]
ignore_letters = ['?','!',',','.']

In [5]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
       word_list = nltk.word_tokenize(pattern)
       words.extend(word_list)
       document.append((word_list,intent["tag"]))
        
       if intent["tag"] not in classes:
            classes.append(intent["tag"])
    
    
    
# document
# words
# classes

In [6]:
# lemmatizer = WordNetLemmatizer()
words = [ lemmatizer.lemmatize(word) for word in words if word not in ignore_letters ]
words = sorted(set(words))
# words
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))        
pickle.dump(classes, open('classes.pkl', 'wb'))

# classes
# words

In [7]:
training = []
outputEmpty = [0] * len(classes)

for doc in document:
    bag = []
    wordPatterns = doc[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(doc[1])] = 1
    training.append(bag+outputRow)

# training
# bag

In [8]:
random.shuffle(training)
training = np.array(training)

# training.shape

# trainX = training[:,0]
# trainY = training[:,1]

trainX = training[:, :len(words)]
trainY = training[:, len(words):]
# len(words)
# trainX

In [9]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [10]:
# model = keras.sequential()

In [11]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
model.fit(trainX, trainY, epochs=200, batch_size=10, verbose=1)
model.save('chatbot.h5')
print('Done')